<a href="https://colab.research.google.com/github/amitgaur95/assignment/blob/main/pyspark_googleplaystore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("PySparkEDA").getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

file_path = "googleplaystore.csv"

readCsv = spark.read.csv(file_path, header=True, inferSchema=True)


# Show the first few rows of the dataset
print('Total no of rows  the dataset :', readCsv.count() )

readCsv.printSchema()

print('Displaying first 10 rows of the dataset')

readCsv.show(10)

Total no of rows  the dataset : 10841
root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

Dsiplaying first 10 rows of the dataset
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------

In [ ]:
print("descriptive anallysis")

readCsv.describe().show()

readCsv = readCsv.drop("Current Ver", "Android Ver" , "Genres")

readCsv.select([count(when(col(x).isNull(), x)).alias(x) for x in readCsv.columns]).show() #finding all null entries

readCsv = readCsv.fillna('Everyone', "Content Rating")


descritive anallysis
+-------+--------------------+-------------+-----------+-----------------+------------------+------------------+-----+------------------+--------------+------+-----------------+-------------+------------------+
|summary|                 App|     Category|     Rating|          Reviews|              Size|          Installs| Type|             Price|Content Rating|Genres|     Last Updated|  Current Ver|       Android Ver|
+-------+--------------------+-------------+-----------+-----------------+------------------+------------------+-----+------------------+--------------+------+-----------------+-------------+------------------+
|  count|               10841|        10841|      10841|            10841|             10841|             10841|10841|             10841|         10840| 10841|            10841|        10840|             10840|
|   mean|                NULL|          1.9|        NaN|444225.1924709356|              NULL| 2.866666666666667|  NaN|               0.

In [ ]:
#renaming, removing M or K string from size column,
#removing '+' symbol from install col,
#removing dollor symbol from price col in below cmds
readCsv = readCsv.withColumnRenamed("Last Updated", "yearUpdated") \

readCsv = readCsv.withColumn("yearUpdated", to_date('yearUpdated','MMM d, yyyy'))


readCsv = readCsv.withColumn('Size', translate('Size', 'Mk', ''))

readCsv = readCsv.withColumn('Installs', translate('Installs', '+,', ''))

readCsv = readCsv.withColumn('Price', translate('Price', '$', ''))

readCsv.show(5)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+-----------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|yearUpdated|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+-----------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|  19|   10000|Free|    0|      Everyone| 2018-01-07|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  14|  500000|Free|    0|      Everyone| 2018-01-15|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 8.7| 5000000|Free|    0|      Everyone| 2018-08-01|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|  25|50000000|Free|    0|          Teen| 2018-06-08|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967| 2.8|  100000|Free|    0|      Everyone| 2018-06-20|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+-----------+
only showing top 5 rows



In [ ]:
#Data type conversion frm string to int, float etc

readCsv = readCsv.withColumn("Installs", readCsv["Installs"].cast(IntegerType()))

readCsv = readCsv.withColumn("Rating", readCsv["Rating"].cast(FloatType()))

readCsv = readCsv.withColumn("Size", readCsv["Size"].cast(FloatType()))

readCsv = readCsv.withColumn("Price", readCsv["Price"].cast(FloatType()))

readCsv.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: float (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- Content Rating: string (nullable = false)
 |-- yearUpdated: date (nullable = true)



In [ ]:
# Applying distinct() to remove duplicate rows
readCsv = readCsv.distinct()
print("Distinct count: "+str(readCsv.count()))


Distinct count: 10358


In [ ]:
#filter condition with selective columns
readCsv.select('App',
 'Category',
 'Size',).filter(readCsv.Rating>='4.5').show()





+--------------------+-------------------+-----+
|                 App|           Category| Size|
+--------------------+-------------------+-----+
|Sync.ME – Caller ...|      COMMUNICATION| 17.0|
| Toca Life: Hospital|          EDUCATION| 24.0|
|Everlance: Free M...|            FINANCE| 14.0|
|Hungry Shark Evol...|               GAME|100.0|
|Fruit Block - Puz...|               GAME| 33.0|
|Breastfeeding Tra...|            MEDICAL| 23.0|
|Taha Al Junaid Qu...|          LIFESTYLE|  4.2|
|FC Barcelona Offi...|             SPORTS| 56.0|
|    BV Taxi - Driver|MAPS_AND_NAVIGATION| 10.0|
|               CF PD|            FINANCE|  2.7|
|Deck Shop for Cla...|             FAMILY| 13.0|
|      Army of Heroes|             FAMILY| 46.0|
|            Checkers|               GAME|  3.3|
|             DN Blog|             SOCIAL|  4.2|
|DX Simulation for...|             FAMILY| 31.0|
|Tunisian Dinar: E...|            FINANCE|  3.8|
| Ec Solutions Mobile|           BUSINESS|  4.0|
|           EG Playe

In [ ]:
#showing Total reviws per category wise using groupBy function

result = readCsv.groupBy("Category") \
            .agg(sum("Reviews").alias("TotalReview"))

result.show(5)

+-------------+------------+
|     Category| TotalReview|
+-------------+------------+
|       EVENTS|    161018.0|
|       SPORTS| 6.5322708E7|
|       COMICS|   3383276.0|
|      WEATHER| 1.4604735E7|
|VIDEO_PLAYERS|1.10380188E8|
+-------------+------------+
only showing top 5 rows



In [ ]:
#showing  the most count of price per app using groupBy and orderBy function

price_count = readCsv.groupBy('Price').count().orderBy('count', ascending=False)
price_count.show()


+------+-----+
| Price|count|
+------+-----+
|   0.0| 9590|
|  0.99|  146|
|  2.99|  125|
|  1.99|   73|
|  4.99|   70|
|  3.99|   60|
|  1.49|   46|
|  5.99|   27|
|  2.49|   25|
|  9.99|   19|
|399.99|   12|
|  6.99|   12|
|  4.49|    9|
| 14.99|    9|
|  7.99|    7|
|  3.49|    7|
|  6.49|    5|
| 19.99|    5|
|  5.49|    5|
| 29.99|    5|
+------+-----+
only showing top 20 rows



In [ ]:
#showing total installations of app per category wise and sorting it based on summation of install

install_trend = readCsv.groupBy('Category').agg({'Installs': 'sum'}).orderBy('sum(Installs)', ascending=False)
install_trend.show()

+-------------------+-------------+
|           Category|sum(Installs)|
+-------------------+-------------+
|               GAME|  31544024415|
|      COMMUNICATION|  24152276251|
|             SOCIAL|  12513867902|
|       PRODUCTIVITY|  12463091369|
|              TOOLS|  11452771915|
|             FAMILY|  10041692505|
|        PHOTOGRAPHY|   9721247655|
|   TRAVEL_AND_LOCAL|   6361887146|
|      VIDEO_PLAYERS|   6222002720|
| NEWS_AND_MAGAZINES|   5393217760|
|           SHOPPING|   2573348785|
|      ENTERTAINMENT|   2455660000|
|    PERSONALIZATION|   2074494782|
|BOOKS_AND_REFERENCE|   1916469576|
|             SPORTS|   1528574498|
| HEALTH_AND_FITNESS|   1360022512|
|           BUSINESS|    863664865|
|            FINANCE|    770348734|
|MAPS_AND_NAVIGATION|    719281890|
|          LIFESTYLE|    534823539|
+-------------------+-------------+
only showing top 20 rows



In [ ]:
#creating temporal view for sql queries

readCsv.createOrReplaceTempView("readCsv")

#sql query for displaying total install per rating range

spark.sql( """
SELECT
    CASE WHEN rating <= 2 THEN '0-2'
         WHEN rating > 2 AND rating <= 3 THEN '2-3'
         WHEN rating > 3 AND rating <= 4 THEN '3-4'
         WHEN rating > 4 AND rating <= 5 THEN '4-5'
    END AS RatingRange,
    SUM(installs) AS TotalInstalls
FROM
    readCsv
GROUP BY RatingRange
ORDER BY RatingRange

""").show()



+-----------+-------------+
|RatingRange|TotalInstalls|
+-----------+-------------+
|       NULL|      5963197|
|        0-2|      6096625|
|        2-3|     54755330|
|        3-4|  17770681630|
|        4-5| 128788417753|
+-----------+-------------+



In [ ]:
# displaying HighestRating, LowestRating, HighestPrice, LowestPrice, NumberOfReviews
# per app wise category

spark.sql("""
SELECT
    Category,
    MAX(Rating) AS HighestRating,
    MIN(Rating) AS LowestRating,
    MAX(Price) AS HighestPrice,
    MIN(Price) AS LowestPrice,
    COUNT(Reviews) AS NumberOfReviews
FROM
    readCsv

WHERE
    Rating >= 1 AND Rating <= 5
GROUP BY
    Category
""").show()



+-------------------+-------------+------------+------------+-----------+---------------+
|           Category|HighestRating|LowestRating|HighestPrice|LowestPrice|NumberOfReviews|
+-------------------+-------------+------------+------------+-----------+---------------+
|             EVENTS|          5.0|         2.9|         0.0|        0.0|             45|
|             SPORTS|          5.0|         1.5|       29.99|        0.0|            286|
|             COMICS|          5.0|         2.8|         0.0|        0.0|             58|
|            WEATHER|          4.8|         3.3|        6.99|        0.0|             75|
|      VIDEO_PLAYERS|          4.9|         1.8|        5.99|        0.0|            160|
|  AUTO_AND_VEHICLES|          4.9|         2.1|        1.99|        0.0|             73|
|          PARENTING|          5.0|         2.0|        4.99|        0.0|             50|
|      ENTERTAINMENT|          4.7|         3.0|        4.99|        0.0|            111|
|    PERSO

In [ ]:
#loading second dataset for app based user reviews

file_path = "googleplaystore_user_reviews.csv"

readReviewCsv = spark.read.csv(file_path, header=True, inferSchema=True)


# Show the first few rows of the dataset
print('Total no of rows  the dataset :', readReviewCsv.count() )

readReviewCsv.printSchema()

print('Dsiplaying first 10 rows of the dataset')

readReviewCsv.show()





Total no of rows  the dataset : 64295
root
 |-- App: string (nullable = true)
 |-- Translated_Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- Sentiment_Polarity: string (nullable = true)
 |-- Sentiment_Subjectivity: string (nullable = true)

Dsiplaying first 10 rows of the dataset
+--------------------+--------------------+--------------------+-------------------+----------------------+
|                 App|   Translated_Review|           Sentiment| Sentiment_Polarity|Sentiment_Subjectivity|
+--------------------+--------------------+--------------------+-------------------+----------------------+
|10 Best Foods for...|"I like eat delic...| also ""Best Befo...|           Positive|                   1.0|
|10 Best Foods for...|This help eating ...|            Positive|               0.25|   0.28846153846153844|
|10 Best Foods for...|                 nan|                 nan|                nan|                   nan|
|10 Best Foods for...|Works great espec

In [ ]:
# Remove rows with NaN values in the specified columns

def to_null(c):
    return when(~(col(c).isNull() | isnan(col(c)) | (trim(col(c)) == "")), col(c))


readReviewCsv.select([to_null(c).alias(c) for c in readReviewCsv.columns]).na.drop().show()

+--------------------+--------------------+--------------------+-------------------+----------------------+
|                 App|   Translated_Review|           Sentiment| Sentiment_Polarity|Sentiment_Subjectivity|
+--------------------+--------------------+--------------------+-------------------+----------------------+
|10 Best Foods for...|"I like eat delic...| also ""Best Befo...|           Positive|                   1.0|
|10 Best Foods for...|This help eating ...|            Positive|               0.25|   0.28846153846153844|
|10 Best Foods for...|Works great espec...|            Positive|                0.4|                 0.875|
|10 Best Foods for...|        Best idea us|            Positive|                1.0|                   0.3|
|10 Best Foods for...|            Best way|            Positive|                1.0|                   0.3|
|10 Best Foods for...|             Amazing|            Positive| 0.6000000000000001|                   0.9|
|10 Best Foods for...|Lookin

In [ ]:
#sql query to display different sentiment percentage bsed on the App

readReviewCsv.createOrReplaceTempView("readReviewCsv")
spark.sql("""
SELECT  App,
    ROUND(SUM(CASE WHEN Sentiment = 'Positive' THEN 1 ELSE 0 END) / COUNT(*) * 100, 0) AS Pos_Percentage,
    ROUND(SUM(CASE WHEN Sentiment = 'Negative' THEN 1 ELSE 0 END) / COUNT(*) * 100, 0) AS Neg_Percentage,
    ROUND(SUM(CASE WHEN Sentiment = 'Neutral' THEN 1 ELSE 0 END) / COUNT(*) * 100, 0) AS Neu_Percentage
FROM readReviewCsv r
WHERE r.sentiment != "nan"
GROUP BY App
""").show(truncate=False)



+-------------------------------------+--------------+--------------+--------------+
|App                                  |Pos_Percentage|Neg_Percentage|Neu_Percentage|
+-------------------------------------+--------------+--------------+--------------+
|BaBe+ - Berita Indonesia             |65.0          |10.0          |25.0          |
|Basketball Stars                     |49.0          |45.0          |4.0           |
|Davis's Drug Guide for Nurses        |67.0          |0.0           |33.0          |
|Find&Save - Local Shopping           |61.0          |23.0          |13.0          |
|Bloomberg Professional               |38.0          |28.0          |35.0          |
|Floor Plan Creator                   |70.0          |8.0           |22.0          |
|Bagan - Myanmar Keyboard             |63.0          |6.0           |28.0          |
|Candy Pop Story                      |55.0          |33.0          |13.0          |
|Cricbuzz - Live Cricket Scores & News|68.0          |29.0       

In [ ]:
#displaying the inner join capability using the two diff dataset

spark.sql("""
SELECT *
FROM readCsv
INNER JOIN readReviewCsv ON readCsv.App = readReviewCsv.App
""").show(5)

+--------------------+------------------+------+-------+----+--------+----+-----+--------------+-----------+--------------------+--------------------+--------------------+------------------+----------------------+
|                 App|          Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|yearUpdated|                 App|   Translated_Review|           Sentiment|Sentiment_Polarity|Sentiment_Subjectivity|
+--------------------+------------------+------+-------+----+--------+----+-----+--------------+-----------+--------------------+--------------------+--------------------+------------------+----------------------+
|10 Best Foods for...|HEALTH_AND_FITNESS|   4.0|   2490| 3.8|  500000|Free|  0.0|  Everyone 10+| 2017-02-17|10 Best Foods for...|"I like eat delic...| also ""Best Befo...|          Positive|                   1.0|
|10 Best Foods for...|HEALTH_AND_FITNESS|   4.0|   2490| 3.8|  500000|Free|  0.0|  Everyone 10+| 2017-02-17|10 Best Foods for...|This help eatin